In [ ]:
# Get Training Data
%pip show nlpaf
%pip install  -e ../

In [1]:
from  iesta.machine_learning.dataloader import load_features_df
import iesta.properties

/home/elba_ro/repos/github/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
from datasets.dataset_dict import DatasetDict


In [1]:
from datasets import load_dataset

imdb = load_dataset("imdb")
type(imdb)

/home/elba_ro/repos/github/conf22-style-transfer/iesta_venv/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /home/elba_ro/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

datasets.dataset_dict.DatasetDict

In [4]:
from iesta.machine_learning.dataloader import IESTAData
def get_training(feature_dfs, ideology, evaluation_classfier_data_flag= IESTAData._WITHOUT_EVAL_CLASSIFIER, discard_okay: bool = True):
    _, feature_dfs = load_features_df()

    training = feature_dfs[ideology]
    print(len(training))

    if evaluation_classfier_data_flag == IESTAData._ONLY_EVAL_CLASSIFIER_:
        training = training[training['is_for_eval_classifier'] == True]
        print(len(training))
    elif evaluation_classfier_data_flag == IESTAData._WITHOUT_EVAL_CLASSIFIER:
        training = training[training['is_for_eval_classifier'] == False]
        print(len(training))
    if discard_okay:
        training = training[training["effect"]!="okay"]
    print(f"Included effects are: {training.effect.unique()}")
    return training

liberal_training = get_training(feature_dfs, "liberal")
liberal_training["binary_effect"].value_counts()
liberal_training.columns.tolist

Included effects are: ['effective' 'ineffective' 'provocative']


<bound method IndexOpsMixin.tolist of Index(['id', 'debate_id', 'p_name', 'effects', 'effect_count', 'top_effect',
       'effect', 'category', 'round', 'argument',
       ...
       'hedge_N_ratio', 'hedge_N_words', 'hedge_dominant', 'toxicity_dominant',
       'toxicity_neutral', 'toxicity_neutral_count', 'toxicity_neutral_ratio',
       'toxicity_toxic', 'toxicity_toxic_count', 'toxicity_toxic_ratio'],
      dtype='object', length=854)>

In [ ]:
from sklearn.model_selection import train_test_split


unique_debates = liberal_training.debate_id.unique()

training_debates, testval_debates = train_test_split(unique_debates, test_size=0.3,
                 random_state=42,
                 shuffle=True)

def _add_splits(row, training_debates, testval_debates):
    if row['debate_id'] in training_debates:
        row["classifier_split"] = "train"
    elif row['debate_id'] in testval_debates:
        row["classifier_split"] = "test"
    else:
        assert False
    return row 

liberal_training = liberal_training.apply(_add_splits, args=(training_debates,testval_debates, ),axis=1)

In [ ]:
liberal_training.classifier_split.value_counts()
import pandas as pd

pd.crosstab(liberal_training.classifier_split, liberal_training.binary_effect)

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
def filter_cols(cols):
    prefices = ["mpqa", "hedge", "emotion", "toxicity", "mpqa", "empath"]
    keep = [c for c in cols if c.split("_")[0] in prefices and not c.endswith("_count")]
    return keep

def filter_feature_df(df: pd.DataFrame, class_col: str = "effect", split_col: str = "classifier_split") : # binary_effect
    feature_cols = filter_cols(df.columns.tolist())
    df_ = df[feature_cols+[class_col, split_col]].copy()
    numeric_cols = df_.select_dtypes(include=np.number).columns.tolist()
    df_ = df_[numeric_cols+[class_col, split_col]]
    
    df_.fillna(0.0, inplace=True)

    # transform y to numeric
    le = LabelEncoder()
    le.fit(df_[class_col])
    df_[class_col] = le.transform(df_[class_col])
    #X = df_.values
    #y_str = df[class_].values


    #def to_numeric(x):
    #    return 0 if x == "ineffective" else (1 if x == "effective" else 2)
    #y = [to_numeric(x) for x in y_str]
    
    return df_, numeric_cols

In [ ]:
le = LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
list(le.classes_)
le.transform(["amsterdam", ])
le.in


In [ ]:


_df, numeric_cols = filter_feature_df(liberal_training, class_col = "binary_effect", split_col = "classifier_split")

In [ ]:
_df.binary_effect.value_counts()

In [ ]:
from nlpaf.ml.feature_based import FeatureBasedML

trainer = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classifier_split", training_cols=numeric_cols, remove_outliers=False, normalize=False)

In [ ]:
dummy = trainer.train_baseline()

In [ ]:
xfold_results = trainer.train_xfold()

In [ ]:
xfold_results.keys()

In [ ]:
xfold_results

In [ ]:
trainer_nooutliers = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classifier_split", training_cols=numeric_cols, remove_outliers=True, normalize=True)

In [ ]:
xfold_nooutlier_results = trainer_nooutliers.train_xfold()

In [ ]:
from nlpaf.ml.feature_based import FeatureBasedML

trainer_ensemble = FeatureBasedML(y_col="binary_effect", dataset=_df, split_label_name="classifier_split", training_cols=numeric_cols, remove_outliers=True, normalize=True)

In [ ]:
from nlpaf.ml import traditional_trainer as tt

results = tt.train_ensemble(trainer_ensemble.X_train, trainer_ensemble.X_test, trainer_ensemble.y_train, trainer_ensemble.y_test)


"""
          precision    recall  f1-score   support

           0       0.06      0.12      0.08       563
           1       0.95      0.91      0.93     11554

    accuracy                           0.87     12117
   macro avg       0.51      0.51      0.50     12117
weighted avg       0.91      0.87      0.89     12117


time: 2222m 7.3s
"""
